# Colab Driver Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

default_path = '/content/drive/My Drive'

Mounted at /content/drive


# Install Chrome Driver
- delete old one and install new one

In [2]:
!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

 This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from Chrome into Chromium for you to use Netflix (or other websites using Encrypted Media Extensions) in Chromium.

1. Download and install Chrome (or extract the necessary files, if

# Import Pakages

In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.webelement import WebElement

from tqdm import tqdm
import pandas as pd
import requests
import cv2

import os
import time
import datetime
from pathlib import Path
from multiprocessing import Pool

In [4]:
selenium.__version__

'4.10.0'

# Crawler Class

In [5]:
class NaverWebtoonCrawler():
    def __init__(self, author_names, author_map):
        self.driver = None
        self.author_names = author_names
        self.author_map = author_map
        self.author_reverse_map = {name: idx for idx, name in self.author_map.items()}

        self.prepare_chrome()
        self.move_to_naverwebtoon()
        # self.login("id", "password")
        # self.move_to_naverwebtoon()

    def prepare_chrome(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')                                      # 메뉴바 같은 불필요한 요소들을 없앤다.
        options.add_argument('--no-sandbox')                                    # 보안 관련 설정 끄기(쿠키 수락 같은 팝업들을 방지하기 위함)
        options.add_argument('--disable-dev-shm-usage')                         # 개발자 도구 끄기.
        options.add_experimental_option('detach', True)                         # 실행이 멈춰도 웹브라우저가 꺼지지 않게 하는 옵션

        service = Service(executable_path=r'/usr/bin/chromedriver')

        driver = webdriver.Chrome(service=service, options=options)
        time.sleep(3)

        self.driver = driver

    def move_to_naverwebtoon(self):
        self.driver.get("https://comic.naver.com/index")
        time.sleep(3)

        print("웹툰 페이지로 이동")

    def login(self, id:str, password:str):
        while True:
            try:
                self.driver.find_element(By.ID, "gnb_login_button").click()     # move to login page
                time.sleep(2)

                print("로그인 페이지 이동")
                break
            except Exception as e:
                print("로그인 페이지 이동 시도...", e)
                continue

        try:
            self.driver.find_element(By.ID, "id").send_keys(id)                 # input id
            self.driver.find_element(By.ID, "pw").send_keys(password)           # input password
            self.driver.find_element(By.ID, "log.login").click()
            time.sleep(3)

            print("로그인 성공!")
        except Exception as e:
            print("로그인 중 오류 발생", e)

    def download_by_author(self, author_name:str, dir_path:Path, time_span:int=5):
        """
        params:
            author_names: ["name", "name", ...]
            time_span: How many years from now

        """
        def search_author(driver:webdriver.Chrome, author_name:str):
            input_xpath = "//*[@id=\"wrap\"]/header/div[1]/div/div[1]/div/input"

            while True:
                try:
                    input_elem = driver.find_element(By.XPATH, input_xpath)
                    input_elem.send_keys(author_name)
                    input_elem.send_keys(Keys.RETURN)
                    time.sleep(1)

                    print(f"{author_name} 검색")
                    break
                except Exception as e:
                    print("작가명 검색 시도...", e)
                    time.sleep(1)
                    continue

            print(f"{author_name}으로 작품 검색")

        def check_works(work:WebElement, author_name:str, year_limit:int) -> bool:
            info_spans = work.find_element(By.CSS_SELECTOR, "div > div.ContentMetaInfo__meta_info--GbTg4").find_elements(By.TAG_NAME, "span")

            flag1 = False
            role1 = info_spans[0].get_attribute("textContent").strip()
            if "글/그림" in role1[:4]:
                flag1 = role1[4:] == author_name

            role2 = info_spans[1].get_attribute("textContent").strip()
            if "그림" in role2[:2]:
                flag1 = role2[2:] == author_name

            year = info_spans[-1].get_attribute("outerText").split("\n")
            flag2 = True if int(year[1].strip().split(".")[0]) > (23-year_limit) else False

            return all([flag1, flag2])

        def get_works(driver:webdriver.Chrome, author_name:str, year_limit:int) -> list:
            work_urls = list()
            work_num = None

            # move page
            page = 1
            while True:
                ul_elem = None
                pagenate_elems = None
                if page == 1:
                    work_num_xpath = "//*[@id=\"content\"]/div[2]/div/div/span"
                    work_num_elem = None
                    while True:
                        try:
                            work_num_elem = driver.find_element(By.XPATH, work_num_xpath)
                            work_num = int(work_num_elem.get_attribute("textContent").split()[1])
                            time.sleep(1)
                            break
                        except Exception as e:
                            time.sleep(1)
                            print(e)
                            continue
                    print(f"{author_name} 검색 결과: 웹툰 {work_num}개")
                    time.sleep(1)

                ul_xpath = "//*[@id=\"content\"]/div[2]/ul"

                if work_num > 5:
                    ul_xpath = "//*[@id=\"content\"]/ul"

                    if page == 1:
                        show_more_xpath = "//*[@id=\"content\"]/div[2]/div/a"
                        driver.find_element(By.XPATH, show_more_xpath).click()

                        time.sleep(1)

                    # page
                    while True:
                        try:
                            pagenate_elems = driver.find_elements(By.CLASS_NAME, "Paginate__page--iRmGj")
                            pagenate_elems[(page%10)-1].click()
                            print(f"{page} 페이지로 이동")
                            break
                        except Exception as e:
                            print(e)
                            time.sleep(1)
                            continue

                while True:
                    try:
                        ul_elem = driver.find_element(By.XPATH, ul_xpath)
                        break
                    except Exception as e:
                        print(e)
                        time.sleep(1)
                        continue

                works = (ul_elem.find_elements(By.TAG_NAME, "li"))
                work_urls += [work.find_element(By.TAG_NAME, "a").get_attribute("href") for work in works if check_works(work, author_name, year_limit)]

                # next button
                if work_num > 5:
                    if int(pagenate_elems[-1].get_attribute("textContent")) == page:
                        next_btn_elem = driver.find_element(By.CLASS_NAME, "Paginate__next--F6rIk")

                        if next_btn_elem.get_attribute("disabled"):
                            break
                        else:
                            next_btn_elem.click()
                            time.sleep(2)
                            print("다음 10개")

                    page += 1

                if work_num < 5:
                    break

            return work_urls

        def download_thumbnail_image(driver:webdriver.Chrome, work_url:WebElement, author_name:str, dir_path:Path, df:pd.DataFrame):
            driver.get(work_url)
            time.sleep(3)

            work_title = None
            while True:
                try:
                    work_title = driver.find_element(By.XPATH, "//*[@id=\"content\"]/div[1]/div/h2").get_attribute("textContent")
                    break
                except Exception as e:
                    print(e)
                    time.sleep(1)
                    continue

            # move page
            page = 1
            while True:
                pagenate_elems = None
                while True:
                    try:
                        pagenate_elems = driver.find_elements(By.CLASS_NAME, "Paginate__page--iRmGj")
                        pagenate_elems[(page%10)-1].click()
                        print(f"{page} 페이지로 이동")
                        break
                    except Exception as e:
                        print(e)
                        time.sleep(1)
                        continue

                episodes_xpath = "//*[@id=\"content\"]/div[3]/ul"
                episodes_ul_elem = None
                while True:
                    try:
                        episodes_ul_elem = driver.find_element(By.XPATH, episodes_xpath)
                        time.sleep(1)
                        break
                    except Exception as e:
                        time.sleep(1)
                        print(e)
                        continue

                episodes = episodes_ul_elem.find_elements(By.TAG_NAME, "li")

                # make directory
                author_dir = dir_path.joinpath("dataset").joinpath(str(self.author_reverse_map[author_name]))
                author_dir.mkdir(parents=True, exist_ok=True)

                # collect thumbnail urls
                thumbnail_urls = list()
                for episode in tqdm(episodes, desc=f"{author_name}: {work_title} url 수집중..."):
                    thumbnail_url = episode.find_element(By.CSS_SELECTOR, "a > div.EpisodeListList__thumbnail_area--EL1aw > img").get_attribute("src")
                    thumbnail_urls.append(thumbnail_url)

                # download from thumbnail's url
                fail_count = 0
                for url in tqdm(thumbnail_urls, desc=f"{author_name}: {work_title} 다운로드중..."):
                    print(url)
                    file_name = url.split("/")[-1]
                    try:
                        # download
                        request = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
                        with open(str(author_dir.joinpath(f"{page}_{file_name}")), "wb") as f:
                            f.write(request.content)
                        time.sleep(1)

                        # write in DataFrame
                        author_index = self.author_reverse_map[author_name]
                        df.loc[df.shape[0]] = [author_name, author_index, str(Path(str(author_index)).joinpath(f"{page}_{file_name}"))]
                    except Exception as e:
                        fail_count += 1
                        time.sleep(1)
                        print(e)
                print(f"{author_name}: {work_title} 다운로드 완료 {fail_count}개 실패")

                # next button
                if int(pagenate_elems[-1].get_attribute("textContent")) == page:
                    next_btn_elem = driver.find_element(By.CLASS_NAME, "Paginate__next--F6rIk")

                    if next_btn_elem.get_attribute("disabled"):
                        break
                    else:
                        next_btn_elem.click()
                        time.sleep(2)
                        print("다음 10개")

                page += 1


        # download_by_author start
        # download
        df = pd.DataFrame(columns=["author", "label", "path"])
        year_limit = int(str(datetime.date.today().year)[-2:]) - time_span

        search_author(self.driver, author_name)

        work_urls = get_works(self.driver, author_name, year_limit)
        for work_url in work_urls:
            download_thumbnail_image(self.driver, work_url, author_name, dir_path, df)

        # write csv
        df.to_csv(str(dir_path.joinpath("dataset").joinpath(str(self.author_reverse_map[author_name])).with_suffix(".csv")))


# Main

In [ ]:
author_names = ["류", "박태준", "삼", "들덤", "락현", "승우"]
map = {idx: name for idx, name in enumerate(set(author_names))}

crawler = NaverWebtoonCrawler(author_names, map)
crawler.download_by_author("들덤", Path("/content/drive/MyDrive/메타버스 아카데미/본과정/Python/data"))

In [7]:
if __name__ == '__main__':
    author_names = ["류", "박태준", "삼", "들덤", "락현", "승우"]
    map = {idx: name for idx, name in enumerate(set(author_names))}

    def crawling(author_name):
        dir_path = Path("/content/drive/MyDrive/메타버스 아카데미/본과정/Python/data")

        crawler = NaverWebtoonCrawler(author_names, map)
        crawler.download_by_author(author_name, dir_path)


    pool = Pool(processes=6)
    pool.map(crawling, author_names)

웹툰 페이지로 이동웹툰 페이지로 이동

웹툰 페이지로 이동
웹툰 페이지로 이동
웹툰 페이지로 이동웹툰 페이지로 이동

락현 검색
락현으로 작품 검색
삼 검색
삼으로 작품 검색
류 검색
류으로 작품 검색
들덤 검색
들덤으로 작품 검색
락현 검색 결과: 웹툰 1개
승우 검색
승우으로 작품 검색
박태준 검색
박태준으로 작품 검색
들덤 검색 결과: 웹툰 1개
삼 검색 결과: 웹툰 49개
류 검색 결과: 웹툰 19개
박태준 검색 결과: 웹툰 20개
승우 검색 결과: 웹툰 3개
글/그림승우 승우
1 페이지로 이동
글/그림최훈 최훈
1 페이지로 이동
글/그림미티 미티
글/그림하일권 하일권
1 페이지로 이동
글/그림혀노글/그림배드이리 1 페이지로 이동 배드이리혀노


글/그림ARI ARI
글/그림류 류


락현: 입학용병 url 수집중...:   0%|          | 0/20 [00:00<?, ?it/s]

글/그림류가명 류가명


락현: 입학용병 url 수집중...:  15%|█▌        | 3/20 [00:00<00:03,  4.40it/s]

글/그림박태준

...

https://image-comic.pstatic.net/webtoon/762071/38/thumbnail_202x120_fa6cfe3b-6ec2-49ee-b498-bd9610f5bc09.jpg


박태준: 욕망일기 다운로드중...:  95%|█████████▍| 18/19 [00:21<00:01,  1.20s/it]

https://image-comic.pstatic.net/webtoon/762071/39/thumbnail_202x120_79a8f0c6-d5b9-4674-bd99-7274f0be3057.jpg


박태준: 욕망일기 다운로드중...: 100%|██████████| 19/19 [00:22<00:00,  1.19s/it]


박태준: 욕망일기 다운로드 완료 0개 실패


자료 모으기 크롤링

이미지 정사각형으로 > 빈자리는 평균색으로 채우기

그 다음은 그냥 학습
- InceptionV3
- InceptionResNetV2
- MobileNetV2
- EfficientNetB7
- ResNet

WandB 적용

네이버 툰필터로 test set 만들기

해치웠나...?

4조